In [ ]:
from LLM import bots
from ASP_Scheduler.problem_descriptions import all_problems
from ASP_Scheduler import scheduler
from ASP_Scheduler import schedulerBaseline
import importlib
from pprint import pprint
from openai import OpenAI
import os



c:\Users\finna\GitHub\SchASPLM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# We illustrate the generation of a scheduling problem using an Inference client, in this case deepseek (using chain-of-thought and few-shot prompting)
# Notice you have to enter your api-key in line 132 of LLM/bots.py
# Notice that delays of 10 seconds have been added in scheduler.py to avoid reaching the token-limit of the inference client.

# Run the LLM scheduler
full_program = scheduler.full_ASP_program(
    all_problems['sports scheduling'],    # Input problem specifications for examination timetabling
    pipe='deepseek',                                  # Input the PIPEline object for the LLM
    printer=True)                              # Set to True to print intermediate outputs           

# Print the full program as it is returned by the scheduler
print('----------------------------FULL PROGRAM----------------------------')
print(full_program)

Instance Template:
team(Team).

venue(Venue).

availability(Venue, Date).


Generator
1 {{ game(Team1, Team2, Venue, Gameday) : team(Team2), venue(Venue), availability(Venue, Gameday), Team1 != Team2 }} 1 :- team(Team1).


Hard Constraints

- No team plays each other more than once at different gamedays.
```
:- game(Team1, Team2, _, Day1), game(Team1, Team2, _, Day2), Day1 != Day2.
```

- Every team plays every other team at least once.
```
:- team(Team1), team(Team2), Team1 != Team2, not game(Team1, Team2, _, _).
```

- No team plays itself.
```
:- game(Team, Team, _, _).
```


Soft Constraints:

-Teams should play at the same venue as little as possible.
```
using_venue(Team,Venue) :- game(Team,_,Venue,_).
penalty("VenueUsage",using_venue(Team,N),(N-1)*1) :- team(Team), N = {{ using_venue(Team,V) }}, N > 1.
```

----------------------------FULL PROGRAM----------------------------

team(Team).

venue(Venue).

availability(Venue, Date).

% Generator

1 {{ game(Team1, Team2, Venue, Game

In [3]:
# We illustrate the generation of a scheduling problem using a local model  (using chain-of-thought and few-shot prompting)
# Notice you have to have your api-key under .env


# If we are working locally, we just instantiate to None and by default we will call the llama 3.1 8B api
CHECKPOINT = None
# CHECKPOINT = "meta-llama/Llama-3.3-70B-Instruct"
# If we work on a cloud, we can use other models like this
PIPE = bots.load_pipe(model_checkpoint=CHECKPOINT, local_dir="./local_models", quantization_config='4bit')

# Examination Timetabling Example

# Run the LLM scheduler
full_program = scheduler.full_ASP_program(
    all_problems['post_enrollment_based_course_time_tabling'],    # Input problem specifications for examination timetabling
    pipe=PIPE,                                  # Input the PIPEline object for the LLM
    printer=False)                              # Set to True to print intermediate outputs           

# Print the full program as it is returned by the scheduler
print('----------------------------FULL PROGRAM----------------------------')
print(full_program)

----------------------------FULL PROGRAM----------------------------

event(Event).

room(Room, Capacity).

timeslot(Day, Hour).

student(Student, Event).

feature_requirement(Event, Feature).

room_feature(Room, Feature).

precedence_constraint(Event1, Event2).

timeslot_constraint(Event, Timeslot).

% Generator

% 1
1 {{ assigned(Event, Timeslot, Room) : room(Room,Capacity), timeslot(Timeslot, _) }} 1 :- event(Event).


% Hard Constraints

% Students must not have overlapping events.
:- assigned(Event1, Timeslot1, Room1), assigned(Event2, Timeslot1, Room2), event(Event1), event(Event2), student(Student, Event1), student(Student, Event2), Room1 != Room2. 
:- assigned(Event1, Timeslot1, Room1), assigned(Event2, Timeslot1, Room1), event(Event1), event(Event2), student(Student, Event1), student(Student, Event2).


% The amount of students in a room at the same time should not exceed the capacity of the room
:- assigned(Event, Timeslot, Room), #count{{Student : student(Student, Event)} : 

In [4]:
# We finally illustrate the generation of a scheduling problem using the baseline model.
# PIPE has to be configured as above

full_program = schedulerBaseline.full_ASP_program(
    all_problems['nurse_scheduling'], 
    pipe=PIPE, 
    printer=False)

print('----------------------------FULL PROGRAM----------------------------')
print(full_program)

----------------------------FULL PROGRAM----------------------------
```asp
# Rule 1: Every nurse is assigned to exactly one shift per day
1 { nurse(X, Y, Z) } 1 :- nurse(X, Z).
:- nurse(X, _, Z), nurse(X, _, Z1), Z != Z1.

# Rule 2: Every day has exactly one shift per nurse
1 { shift(Y, Z, X) } 1 :- shift(Y, X, Z).
:- shift(Y, X, Z), shift(Y1, X, Z), Y != Y1.

# Rule 3: Every day has all shift types
1 { shift(X, Y, Z) } 1 :- shift(X, Y).

# Rule 4: Every nurse has the same number of shifts per day
1 { shift(X, Y, Z) } 1 :- shift(X, Y).

# Rule 5: Every nurse works at least the minimum number of hours
:- nurse(X, Y, Z), work_requirements(Y, M1, _), sum{ shift(X, _, D) : D =< Z, nurse(X, _, D) } < M1.

# Rule 6: Every nurse works at most the maximum number of hours
:- nurse(X, Y, Z), work_requirements(_, _, M2), sum{ shift(X, _, D) : D =< Z, nurse(X, _, D) } > M2.

# Rule 7: Nurses have exactly 30 days of vacation
:- nurse(X, _, Z), shift(X, vacation, Z), #count{ Z : shift(X, vacation, 